In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import os
import json

import nibabel
import nilearn
import numpy as np
import sklearn
import pandas as pd

from nilearn import plotting

import rtcloud

In [ ]:
# Prepare test samples
test_subject = 2
data_dir = "data"
os.system('mkdir -p %s' % data_dir)

haxby = nilearn.datasets.fetch_haxby(data_dir="datasets", subjects=[test_subject])
labels_test = pd.read_csv(haxby.session_target[0], sep=" ")
y_test_all = labels_test["labels"]
condition_mask_test = y_test_all.isin(["face", "house"])

In [ ]:
fmri_img_test = nilearn.image.index_img(haxby.func[0], condition_mask_test)
fmri_data_test = fmri_img_test.get_data()
y_test = y_test_all[condition_mask_test]
for sample in range(fmri_data_test.shape[-1]):
    sample_data = fmri_data_test[..., sample][..., np.newaxis]
    sample_img = nilearn.image.new_img_like(fmri_img_test, sample_data)
    nibabel.save(sample_img, "{}/subject-{}-sample-{:03}.nii.gz".format(data_dir, test_subject, sample))

In [ ]:
conf = {
    'test': 1
}
mask_img = nibabel.load(haxby.mask)
conf['experiment_data'] = {
    "models": sklearn.externals.joblib.load("models.pkl"),
    "mask_data": mask_img.get_data().astype(bool),
    "mask_affine": mask_img.affine,
}

# client = rtcloud.Client(server_ip='ec2-34-237-207-176.compute-1.amazonaws.com', conf=conf)
client = rtcloud.Client(server_ip='localhost', conf=conf)

client.start()

In [ ]:
client.queue(input_dir=data_dir, tr=2500, loop=True)
client.watch()
client.display()